In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/stdcode2024/Session07 - 유형별로 실습하는 머신러닝/middleexam_김지수/codeset

/content/drive/MyDrive/Colab Notebooks/stdcode2024/Session07 - 유형별로 실습하는 머신러닝/middleexam_김지수/codeset


In [70]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import ensemble
from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt
import pickle

In [4]:
ucData = pd.read_csv("../dataset/usedCar_data.csv")
ucData
# ucData.shape # row 6019개, column 14개

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,6014,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,7.88 Lakh,4.75
6015,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,NaN,4.00
6016,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,NaN,2.90
6017,6017,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,NaN,2.65


# A. 데이터 전처리

In [5]:
# 의미 없는 column 삭제
ucData = ucData.drop(["Unnamed: 0", "New_Price"], axis=1)

In [6]:
ucData # row 6019개, column 12개

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
...,...,...,...,...,...,...,...,...,...,...,...,...
6014,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,4.75
6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,4.00
6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,2.90
6017,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,2.65


In [7]:
# null 포함하는 컬럼 찾기
columnsWithNull = [col for col in ucData.columns if ucData[col].isnull().any()]
columnsWithNull

['Mileage', 'Engine', 'Power', 'Seats']

In [8]:
# NaN 포함하는 row 삭제
ucData = ucData.dropna(axis=0, inplace=False).reset_index(drop=True)

In [9]:
ucData # row 5975개, column 12개

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
...,...,...,...,...,...,...,...,...,...,...,...,...
5970,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,4.75
5971,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,4.00
5972,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,2.90
5973,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,2.65


In [10]:
# 차 종류 개수
len(np.unique(list(ucData.Name))) # 1885개! 너무 많음 => 맨 앞 첫글자인 브랜드명만 남겨보자

1855

In [11]:
# 차 브랜드명만 남기기
names = list(ucData.Name)
# names[0].split(' ', 1)[0]

for i in range(0, len(names)):
  names[i] = names[i].split(' ', 1)[0]

# names
ucData.Name = names

In [12]:
# 브랜드명만 남긴 차 종류 개수
len(np.unique(list(ucData.Name))) # 31개!

31

In [14]:
# Mileage, Engine, Power 컬럼의 단위 문자(km/kg, cc, bhp) 삭제
mileage = list(ucData.Mileage)
engine = list(ucData.Engine)
power = list(ucData.Power)

for i in range(len(names)):
  mileage[i] = mileage[i].split(' ', 1)[0]
  engine[i] = engine[i].split(' ', 1)[0]
  power[i] = power[i].split(' ', 1)[0]

ucData.Mileage = mileage
ucData.Engine = engine
ucData.Power = power
ucData.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti,Mumbai,2010,72000,CNG,Manual,First,26.6,998,58.16,5.0,1.75
1,Hyundai,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,12.50
2,Honda,Chennai,2011,46000,Petrol,Manual,First,18.2,1199,88.7,5.0,4.50
3,Maruti,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00
4,Audi,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968,140.8,5.0,17.74


### A-1. 타입 변환



In [15]:
# 밑에서 float으로 타입 통합할 때 Power 컬럼에 'null' 이 있어서 불가능하다는 에러 뜸!
# 테스트 결과 뒤에 붙는 단위 문자를 삭제한 후에 문제가 발생하는 듯
ucData.Power = ucData.Power.replace('null', np.nan)

# NaN 포함하는 row 삭제
ucData = ucData.dropna(axis=0, inplace=False).reset_index(drop=True)
ucData.shape # row 5872개, column 12개

(5872, 12)

In [17]:
# Mileage, Engine, Power 컬럼을 숫자 타입으로 변환
ucData.Mileage = ucData.Mileage.astype(float)
ucData.Engine = ucData.Engine.astype(int)
ucData.Power = ucData.Power.astype(float)

### A-1. 특성 숫자컬럼 추가 (라벨 인코딩)

In [18]:
np.unique(list(ucData.Transmission)) # Transmission의 종류는 'Manual', 'Automatic' 으로 2개

array(['Automatic', 'Manual'], dtype='<U9')

In [19]:
np.unique(list(ucData.Owner_Type)) # Owner_Type의 종류는 'First', 'Second', Third', 'Fourth & Above' 로 4개

array(['First', 'Fourth & Above', 'Second', 'Third'], dtype='<U14')

In [20]:
np.unique(list(ucData.Fuel_Type)) # Fuel_Type의 종류는 'CNG', 'Diesel', 'LPG', 'Petrol' 로 4개

array(['CNG', 'Diesel', 'LPG', 'Petrol'], dtype='<U6')

In [21]:
maLabelEn = LabelEncoder()
# otLabelEn = LabelEncoder()
ucData["LE_Transmission"] = maLabelEn.fit_transform( ucData.Transmission )
# ucData["LE_Owner_Type"] = otLabelEn.fit_transform( ucData.Owner_Type )

In [22]:
otLabelDict = {"First":0,
               "Second":1,
               "Third":2,
               "Fourth & Above":3}
ftLabelDict = {"Petrol":0,
               "LPG":1,
               "Diesel":2,
               "CNG":3}
ucData["LE_Owner_Type"] = ucData.Owner_Type.map(otLabelDict)
ucData["LE_Fuel_Type"] = ucData.Fuel_Type.map(ftLabelDict)

In [23]:
ucData

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,LE_Transmission,LE_Owner_Type,LE_Fuel_Type
0,Maruti,Mumbai,2010,72000,CNG,Manual,First,26.60,998,58.16,5.0,1.75,1,0,3
1,Hyundai,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5.0,12.50,1,0,2
2,Honda,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5.0,4.50,1,0,0
3,Maruti,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00,1,0,2
4,Audi,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5.0,17.74,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,Maruti,Delhi,2014,27365,Diesel,Manual,First,28.40,1248,74.00,5.0,4.75,1,0,2
5868,Hyundai,Jaipur,2015,100000,Diesel,Manual,First,24.40,1120,71.00,5.0,4.00,1,0,2
5869,Mahindra,Jaipur,2012,55000,Diesel,Manual,Second,14.00,2498,112.00,8.0,2.90,1,1,2
5870,Maruti,Kolkata,2013,46000,Petrol,Manual,First,18.90,998,67.10,5.0,2.65,1,0,0


### A-2. 특성 선정

In [25]:
corrDf = ucData.corr(numeric_only=True)
corrDf

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,LE_Transmission,LE_Owner_Type,LE_Fuel_Type
Year,1.000000,-0.169369,0.285623,-0.068045,0.014531,0.007833,0.299475,-0.085406,-0.386030,0.113159
Kilometers_Driven,-0.169369,1.000000,-0.060608,0.093030,0.033490,0.083072,-0.008249,0.024111,0.080425,0.104494
Mileage,0.285623,-0.060608,1.000000,-0.637258,-0.538844,-0.331576,-0.341652,0.368398,-0.148770,0.143553
Engine,-0.068045,0.093030,-0.637258,1.000000,0.866301,0.401116,0.658047,-0.501928,0.058705,0.399198
Power,0.014531,0.033490,-0.538844,0.866301,1.000000,0.101460,0.772843,-0.644688,0.026002,0.262619
Seats,0.007833,0.083072,-0.331576,0.401116,0.101460,1.000000,0.055547,0.074554,0.011524,0.297182
Price,0.299475,-0.008249,-0.341652,0.658047,0.772843,0.055547,1.000000,-0.585623,-0.091098,0.301626
LE_Transmission,-0.085406,0.024111,0.368398,-0.501928,-0.644688,0.074554,-0.585623,1.000000,0.002313,-0.119206
LE_Owner_Type,-0.386030,0.080425,-0.148770,0.058705,0.026002,0.011524,-0.091098,0.002313,1.000000,-0.041181
LE_Fuel_Type,0.113159,0.104494,0.143553,0.399198,0.262619,0.297182,0.301626,-0.119206,-0.041181,1.000000


In [26]:
stdRatio = 0.5

In [32]:
# 문제지
features = list( corrDf.loc[ (abs(corrDf.Price) >= stdRatio) &
                            (corrDf.Price != 1) ].index )
features

['Engine', 'Power', 'LE_Transmission']

In [31]:
# 정답지
label = ["Price"]

### A-2. 데이터 분리

In [35]:
trainingDataFeatures,\
testDataFeatures,\
trainingDataLabel,\
testDataLabel = train_test_split( ucData.loc[:,features],
                                  ucData.loc[:,label],
                                  test_size=0.2,
                                  random_state=10)

In [38]:
# 인덱스 리셋
trainingDataFeatures = trainingDataFeatures.reset_index(drop=True)
testDataFeatures = testDataFeatures.reset_index(drop=True)
trainingDataLabel = trainingDataLabel.reset_index(drop=True)
testDataLabel = testDataLabel.reset_index(drop=True)

In [42]:
# 8:2 비율 체크
print(trainingDataFeatures.shape)
print(testDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataLabel.shape)

(4697, 3)
(1175, 3)
(4697, 1)
(1175, 1)


In [44]:
# trainingDataFeatures

# B. 모델 적용

### Decision Tree

In [45]:
modelDt = tree.DecisionTreeRegressor(random_state=10)

In [56]:
fittedDtModel = modelDt.fit( X=trainingDataFeatures,
                             y=trainingDataLabel )

### Random Forest

In [50]:
modelRf = ensemble.RandomForestRegressor(random_state=10)

In [51]:
fittedRfModel = modelRf.fit( X=trainingDataFeatures,
                             y=trainingDataLabel )

<ipython-input-51-ac009ac7f0cd>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fittedRfModel = modelRf.fit( X=trainingDataFeatures,


# C. 예측

### Decision Tree

In [57]:
predictValueDt = fittedDtModel.predict(X=testDataFeatures)

array([ 9.74      ,  1.82333333,  4.525     , ...,  3.76575758,
        6.7       , 35.81133333])

In [59]:
testDataLabel["Predict_DT"] = predictValueDt
testDataLabel

,Price,Predict_DT
0,9.00,9.740000
1,2.00,1.823333
2,4.40,4.525000
3,51.69,36.570000
4,5.50,6.033500
...,...,...
1170,8.85,8.374588
1171,2.49,3.765758
1172,3.80,3.765758
1173,6.50,6.700000


### Random Forest

In [60]:
predictValueRf = fittedRfModel.predict(X=testDataFeatures)

In [62]:
testDataLabel["Predict_RF"] = predictValueRf
testDataLabel

,Price,Predict_DT,Predict_RF
0,9.00,9.740000,10.114430
1,2.00,1.823333,1.820186
2,4.40,4.525000,4.524865
3,51.69,36.570000,37.125075
4,5.50,6.033500,6.032184
...,...,...,...
1170,8.85,8.374588,8.374352
1171,2.49,3.765758,3.758929
1172,3.80,3.765758,3.758929
1173,6.50,6.700000,6.683616


# 2개 모델 정확도 비교해보기

In [74]:
testDataFeatures

,Engine,Power,LE_Transmission
0,2393,163.5,1
1,1388,68.0,1
2,1896,106.0,1
3,4134,335.2,0
4,1498,98.6,1
...,...,...,...
1170,1248,88.5,1
1171,1199,73.9,1
1172,1199,73.9,1
1173,2499,141.0,1


In [67]:
testDataLabel["DT_ACC"] = 1 - abs(testDataLabel.Price - testDataLabel.Predict_DT) / testDataLabel.Price
testDataLabel["RF_ACC"] = 1 - abs(testDataLabel.Price - testDataLabel.Predict_RF) / testDataLabel.Price
testDataLabel

,Price,Predict_DT,Predict_RF,DT_ACC,RF_ACC
0,9.00,9.740000,10.114430,0.917778,0.876174
1,2.00,1.823333,1.820186,0.911667,0.910093
2,4.40,4.525000,4.524865,0.971591,0.971622
3,51.69,36.570000,37.125075,0.707487,0.718225
4,5.50,6.033500,6.032184,0.903000,0.903239
...,...,...,...,...,...
1170,8.85,8.374588,8.374352,0.946281,0.946254
1171,2.49,3.765758,3.758929,0.487648,0.490390
1172,3.80,3.765758,3.758929,0.990989,0.989192
1173,6.50,6.700000,6.683616,0.969231,0.971751


In [69]:
print( testDataLabel.DT_ACC.mean() )
print( testDataLabel.RF_ACC.mean() )

0.7182738472165704
0.7165481581179222


# 피클로 저장

In [71]:
with open("./mldtcore.dump", "wb") as fw:
  pickle.dump(fittedDtModel, fw)

with open("./mlrfcore.dump", "wb") as fw:
  pickle.dump(fittedRfModel, fw)